In [191]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

day = 0
file_name = f"./round-2-island-data-bottle/prices_round_2_day_{day}.csv"
clean_log = f"./clean_log/orchids.csv"
df = pd.read_csv(file_name, sep=';')
clean_df = pd.read_csv(clean_log, sep=',')


In [192]:
clean_df['implied_bid'] = clean_df['bid_price'] - clean_df['exportTariff'] - clean_df['transportFees'] - 0.1
clean_df['implied_ask'] = clean_df['ask_price'] + clean_df['importTariff'] + clean_df['transportFees']
clean_df['implied_mid'] = (clean_df['implied_bid'] + clean_df['implied_ask']) / 2

In [193]:
df

,timestamp,ORCHIDS,TRANSPORT_FEES,EXPORT_TARIFF,IMPORT_TARIFF,SUNLIGHT,HUMIDITY,DAY
0,0,1111.75,1.5,10.5,-2.0,4500.0000,95.000000,0
1,100,1111.75,1.5,10.5,-2.0,4500.0356,94.995990,0
2,200,1109.75,1.5,10.5,-2.0,4500.0700,94.991980,0
3,300,1109.25,1.5,10.5,-2.0,4500.1016,94.987970,0
4,400,1110.75,1.5,10.5,-2.0,4500.1313,94.983950,0
...,...,...,...,...,...,...,...,...
9996,999600,988.25,1.3,10.5,-2.6,1703.5710,64.989100,0
9997,999700,988.00,1.3,10.5,-2.6,1702.6835,64.991806,0
9998,999800,988.75,1.3,10.5,-2.6,1701.7925,64.994530,0
9999,999900,986.75,1.3,10.5,-2.6,1700.8980,64.997260,0


In [194]:
df["SUNLIGHT_ENOUGH"] = df.apply(lambda x: 1 if x['SUNLIGHT'] >= 2500 else 0, axis = 1)
df["SUNLIGHT_ENOUGH_PERCENTAGE"] = df.apply(lambda x: (df.loc[df['timestamp'] <= x['timestamp'], 'SUNLIGHT_ENOUGH'].sum() / df.loc[df['timestamp'] <= x['timestamp'], 'SUNLIGHT_ENOUGH'].count()) * 100, axis=1)
df["TEST"] = 100 * 7/12 - df['SUNLIGHT_ENOUGH_PERCENTAGE']
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['TEST'], name='TEST', mode='lines'))
fig.add_trace(go.Scatter(x=df['timestamp'], y=df['ORCHIDS'], name='ORCHIDS', mode='lines', yaxis='y2'))
fig.update_layout(yaxis2=dict(overlaying='y', side='right'))
fig.show()



In [188]:
px.line(df, x='timestamp', y='HUMIDITY')



In [178]:

# df['SUNLIGHT_PAST_AVG'] = df['SUNLIGHT']
# df["PROD_RED_SUN"] = df.apply(lambda x: 0 if x["SUNLIGHT_HR"] >= 7 else abs(x["SUNLIGHT_HR"] - 7)*6*4 , axis=1)
# df["PROD_RED_HUM"] = df.apply(lambda x: 0 if x["HUMIDITY"] <= 80 and x["HUMIDITY"] >= 60 else abs(80 - x["HUMIDITY"])/5 * 2 if x["HUMIDITY"] > 80 else abs(60 - x["HUMIDITY"])/5 * 2, axis=1)
# df['PROD'] = 100 - df['PROD_RED_SUN'] - df['PROD_RED_HUM']
fig = go.Figure()
fig.add_trace(go.Scatter(x=clean_df['timestamp'], y=clean_df['implied_mid'], name='implied_mid', mode='lines'))
fig.add_trace(go.Scatter(x=clean_df['timestamp'], y=clean_df['mid_price'], name='mid_price', mode='lines', yaxis='y2'))
fig.update_layout(yaxis2=dict(overlaying='y', side='right'))
fig.show()


In [163]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=clean_df['timestamp'], y=clean_df['mid_price'], name='mid_price', mode='lines'))
fig.add_trace(go.Scatter(x=clean_df['timestamp'], y=clean_df['bid_price'], name='bid_price', mode='lines'))
fig.add_trace(go.Scatter(x=clean_df['timestamp'], y=clean_df['ask_price'], name='ask_price', mode='lines'))
fig.add_trace(go.Scatter(x=clean_df['timestamp'], y=clean_df['ask_price_1'], name='ask_price_1', mode='lines'))

fig.show()


In [162]:
px.line(clean_df, x='timestamp', y='importTariff', title='importTariff')




In [135]:
fig = go.Figure()
columns = ["bid_price", "transportFees", "importTariff", "ask_price"]

fig.add_trace(go.Scatter(x=df['timestamp'], y=df[columns[0]], name=columns[0]))

fig.add_trace(go.Scatter(x=df['timestamp'], y=df[columns[1]], name=columns[1], yaxis='y2'))

fig.add_trace(go.Scatter(x=df['timestamp'], y=df[columns[2]], name=columns[2], yaxis='y3'))

fig.add_trace(go.Scatter(x=df['timestamp'], y=df[columns[3]], name=columns[3], yaxis='y4'))

# fig.add_trace(go.Scatter(x=df['timestamp'], y=df[columns[4]], name=columns[4], yaxis='y4'))

# fig.add_trace(go.Scatter(x=df['timestamp'], y=df[columns[5]], name=columns[5], yaxis='y5'))

fig.update_layout(
    yaxis=dict(range=[df[columns[0]].min(), df[columns[0]].max()]),
    yaxis2=dict(range=[df[columns[1]].min(), df[columns[1]].max()], overlaying="y", side="right"),
    yaxis3=dict(range=[df[columns[2]].min(), df[columns[2]].max()], overlaying="y", side="right", anchor="free", autoshift=True),
    yaxis4=dict(range=[df[columns[3]].min(), df[columns[3]].max()], overlaying="y", anchor="free", autoshift=True),
    # yaxis5=dict(range=[df[columns[5]].min(), df[columns[5]].max()], anchor="free", overlaying="y", autoshift=True),
)

fig.show()

In [119]:
df['HUMIDITY_DIFF'] = abs(df["HUMIDITY"] - 70)
df_hum_diff = df[df["HUMIDITY_DIFF"] >= 10].copy()
df_hum_diff.reset_index(inplace=True)
df_hum_diff["HUMIDITY_RET"] = (df_hum_diff.shift(100)["HUMIDITY_DIFF"] - df_hum_diff["HUMIDITY_DIFF"])/df_hum_diff["HUMIDITY_DIFF"]
print(df_hum_diff['HUMIDITY_RET'].mean())
print(df_hum_diff['HUMIDITY_RET'].std())
print(df_hum_diff['HUMIDITY_RET'].min())
print(df_hum_diff['HUMIDITY_RET'].max())



0.03279080507981937
0.03120047455646335
-0.0018464427289793105
0.13903216777624508
